In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np

# Функции для расчета NDCG
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0

def ndcg_at_k(r, k):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0
    return dcg_at_k(r, k) / dcg_max

def calculate_ndcg(df, k):
    ndcg_scores = []
    for _, group in df.groupby('search_id'):
        relevance = group['target'].values
        ndcg_scores.append(ndcg_at_k(relevance, k))
    return np.mean(ndcg_scores)

# Загрузка данных
train_data = pd.read_csv("train_df.csv")
test_data = pd.read_csv("test_df.csv")

# Подготовка данных
X_train = train_data.drop(columns=["search_id", "target"])
y_train = train_data["target"]

X_test = test_data.drop(columns=["search_id", "target"])
y_test = test_data["target"]

# Разделение данных на обучающий и валидационный наборы
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Инициализация и обучение модели
model = XGBClassifier()
model.fit(X_train, y_train)

# Предсказание на валидационном наборе и оценка качества модели
y_pred_proba = model.predict_proba(X_valid)[:, 1]
auc_score = roc_auc_score(y_valid, y_pred_proba)
print(f"ROC AUC на валидационном наборе: {auc_score}")

# Предсказание на тестовом наборе
y_test_pred_proba = model.predict_proba(X_test)[:, 1]

# Оценка качества модели на тестовом наборе, если есть соответствующие метки
test_auc_score = roc_auc_score(y_test, y_test_pred_proba)
print(f"ROC AUC на тестовом наборе: {test_auc_score}")

# Расчет NDCG на тестовом наборе
ndcg_10 = calculate_ndcg(test_data, k=10)
print(f"NDCG@10 на тестовом наборе: {ndcg_10}")


ROC AUC на валидационном наборе: 0.6916701079582326
ROC AUC на тестовом наборе: 0.6512886090891206
NDCG@10 на тестовом наборе: 0.10491545769332042
